In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import gc
from tqdm import tqdm

In [4]:
import pandas as pd

def set_table_dtypes(df):
    for col in df.columns:
        if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
            df[col] = df[col].astype('Int64')
        elif col in ["date_decision"]:
            df[col] = pd.to_datetime(df[col])
        elif col[-1] in ("P", "A"):
            df[col] = df[col].astype(float)
        elif col[-1] in ("D",):
            df[col] = pd.to_datetime(df[col])
    return df

def convert_strings(df):
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            df[col] = df[col].astype(pd.CategoricalDtype(categories=new_categories, ordered=True))
    return df


# train base

In [5]:
train_base = pd.read_parquet('/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train/train_base.parquet')
train_base.drop(columns=['date_decision','MONTH','WEEK_NUM'],inplace=True)
train_base.head()

,case_id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,1


In [4]:
file_for_open = ['train_tax_registry_c_1.parquet',
 'train_static_0_0.parquet',
 'train_applprev_1_1.parquet',
 'train_static_cb_0.parquet',
 'train_tax_registry_a_1.parquet',
 'train_tax_registry_b_1.parquet',
 'train_deposit_1.parquet',
 'train_static_0_1.parquet',
 'train_applprev_1_0.parquet']

In [10]:
feature_list = ['case_id','actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'avgdbdtollast24m_4525197P', 'avgdpdtolclosure24_3658938P', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgmaxdpdlast9m_3716943P', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'clientscnt12m_3712952L', 'clientscnt3m_3712950L', 'clientscnt6m_3712949L', 'clientscnt_100L', 'clientscnt_1022L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_136L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'daysoverduetolerancedd_3976961L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'downpmt_116A', 'eir_270L', 'homephncnt_628L', 'inittransactionamount_650A', 'interestrate_311L', 'interestrategrace_34L', 'lastapprcredamount_781A', 'lastdependentsnum_448L', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcredamount_222A', 'maininc_215A', 'mastercontrelectronic_519L', 'mastercontrexist_109L', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdbddpdlast1m_3658939P', 'maxdbddpdtollast12m_3658940P', 'maxdbddpdtollast6m_4187119P', 'maxdebt4_972A', 'maxdpdfrom6mto36m_3546853P', 'maxdpdinstlnum_3546846P', 'maxdpdlast12m_727P', 'maxdpdlast24m_143P', 'maxdpdlast3m_392P', 'maxdpdlast6m_474P', 'maxdpdlast9m_1059P', 'maxdpdtolerance_374P', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'mindbddpdlast24m_3658935P', 'mindbdtollast24m_4525191P', 'mobilephncnt_593L', 'monthsannuity_845L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numincomingpmts_3546848L', 'numinstlallpaidearly3d_817L', 'numinstls_657L', 'numinstlsallpaid_934L', 'numinstlswithdpd10_728L', 'numinstlswithdpd5_4187116L', 'numinstlswithoutdpd_562L', 'numinstmatpaidtearly2d_4499204L', 'numinstpaid_4499208L', 'numinstpaidearly3d_3546850L', 'numinstpaidearly3dest_4493216L', 'numinstpaidearly5d_1087L', 'numinstpaidearly5dest_4493211L', 'numinstpaidearly5dobd_4499205L', 'numinstpaidearly_338L', 'numinstpaidearlyest_4493214L', 'numinstpaidlastcontr_4325080L', 'numinstpaidlate1d_3546852L', 'numinstregularpaid_973L', 'numinstregularpaidest_4493210L', 'numinsttopaygr_769L', 'numinsttopaygrest_4493213L', 'numinstunpaidmax_3546851L', 'numinstunpaidmaxest_4493212L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'pctinstlsallpaidearl3d_427L', 'pctinstlsallpaidlat10d_839L', 'pctinstlsallpaidlate1d_3546856L', 'pctinstlsallpaidlate4d_3546849L', 'pctinstlsallpaidlate6d_3546844L', 'pmtnum_254L', 'posfpd10lastmonth_333P', 'posfpd30lastmonth_3976960P', 'posfstqpd30lastmonth_3976962P', 'price_1097A', 'sellerplacecnt_915L', 'sellerplacescnt_216L', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallast1m_4525188A', 'actualdpd_943P', 'annuity_853A', 'byoccupationinc_3656910L', 'childnum_21L', 'credacc_actualbalance_314A', 'credacc_credlmt_575A', 'credacc_maxhisbal_375A', 'credacc_minhisbal_90A', 'credacc_transactions_402L', 'credamount_590A', 'currdebt_94A', 'downpmt_134A', 'mainoccupationinc_437A', 'maxdpdtolerance_577P', 'outstandingdebt_522A', 'pmtnum_8L', 'revolvingaccount_394A', 'tenor_203L', 'contractssum_5085716L', 'days120_123L', 'days180_256L', 'days30_165L', 'days360_512L', 'days90_310L', 'firstquarter_103L', 'for3years_128L', 'for3years_504L', 'for3years_584L', 'formonth_118L', 'formonth_206L', 'formonth_535L', 'forquarter_1017L', 'forquarter_462L', 'forquarter_634L', 'fortoday_1092L', 'forweek_1077L', 'forweek_528L', 'forweek_601L', 'foryear_618L', 'foryear_818L', 'foryear_850L', 'fourthquarter_440L', 'numberofqueries_373L', 'pmtaverage_3A', 'pmtaverage_4527227A', 'pmtaverage_4955615A', 'pmtcount_4527229L', 'pmtcount_4955617L', 'pmtcount_693L', 'pmtscount_423L', 'pmtssum_45A', 'secondquarter_766L', 'thirdquarter_1082L', 'amount_4527230A', 'amount_4917619A', 'amount_416A', 'actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'avgdbdtollast24m_4525197P', 'avgdpdtolclosure24_3658938P', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgmaxdpdlast9m_3716943P', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'clientscnt12m_3712952L', 'clientscnt3m_3712950L', 'clientscnt6m_3712949L', 'clientscnt_100L', 'clientscnt_1022L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_136L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'daysoverduetolerancedd_3976961L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'downpmt_116A', 'eir_270L', 'homephncnt_628L', 'inittransactionamount_650A', 'interestrate_311L', 'interestrategrace_34L', 'lastapprcredamount_781A', 'lastdependentsnum_448L', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcredamount_222A', 'maininc_215A', 'mastercontrelectronic_519L', 'mastercontrexist_109L', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdbddpdlast1m_3658939P', 'maxdbddpdtollast12m_3658940P', 'maxdbddpdtollast6m_4187119P', 'maxdebt4_972A', 'maxdpdfrom6mto36m_3546853P', 'maxdpdinstlnum_3546846P', 'maxdpdlast12m_727P', 'maxdpdlast24m_143P', 'maxdpdlast3m_392P', 'maxdpdlast6m_474P', 'maxdpdlast9m_1059P', 'maxdpdtolerance_374P', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'mindbddpdlast24m_3658935P', 'mindbdtollast24m_4525191P', 'mobilephncnt_593L', 'monthsannuity_845L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numincomingpmts_3546848L', 'numinstlallpaidearly3d_817L', 'numinstls_657L', 'numinstlsallpaid_934L', 'numinstlswithdpd10_728L', 'numinstlswithdpd5_4187116L', 'numinstlswithoutdpd_562L', 'numinstmatpaidtearly2d_4499204L', 'numinstpaid_4499208L', 'numinstpaidearly3d_3546850L', 'numinstpaidearly3dest_4493216L', 'numinstpaidearly5d_1087L', 'numinstpaidearly5dest_4493211L', 'numinstpaidearly5dobd_4499205L', 'numinstpaidearly_338L', 'numinstpaidearlyest_4493214L', 'numinstpaidlastcontr_4325080L', 'numinstpaidlate1d_3546852L', 'numinstregularpaid_973L', 'numinstregularpaidest_4493210L', 'numinsttopaygr_769L', 'numinsttopaygrest_4493213L', 'numinstunpaidmax_3546851L', 'numinstunpaidmaxest_4493212L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'pctinstlsallpaidearl3d_427L', 'pctinstlsallpaidlat10d_839L', 'pctinstlsallpaidlate1d_3546856L', 'pctinstlsallpaidlate4d_3546849L', 'pctinstlsallpaidlate6d_3546844L', 'pmtnum_254L', 'posfpd10lastmonth_333P', 'posfpd30lastmonth_3976960P', 'posfstqpd30lastmonth_3976962P', 'price_1097A', 'sellerplacecnt_915L', 'sellerplacescnt_216L', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallast1m_4525188A', 'actualdpd_943P', 'annuity_853A', 'byoccupationinc_3656910L', 'childnum_21L', 'credacc_actualbalance_314A', 'credacc_credlmt_575A', 'credacc_maxhisbal_375A', 'credacc_minhisbal_90A', 'credacc_transactions_402L', 'credamount_590A', 'currdebt_94A', 'downpmt_134A', 'mainoccupationinc_437A', 'maxdpdtolerance_577P', 'outstandingdebt_522A', 'pmtnum_8L', 'revolvingaccount_394A', 'tenor_203L']

In [ ]:
"""
train_list_tax = ['train_tax_registry_c_1.parquet', 'train_tax_registry_a_1.parquet','train_tax_registry_b_1.parquet']
train_list_app = ['train_applprev_1_1.parquet', 'train_applprev_1_0.parquet']
train_list_static = ['train_static_0_0.parquet','train_static_0_1.parquet','train_static_cb_0.parquet']
"""

# Part Static

In [11]:
dataPath = '/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train'
trian_sta_path = [
    dataPath + "/train_static_0_0.parquet",
    dataPath + "/train_static_0_1.parquet",
] 
train_static = pd.concat(
    [pd.read_parquet(file_path).pipe(set_table_dtypes) for file_path in trian_sta_path],
    axis=0,  # Concatenate along the rows (vertically)
    ignore_index=True  # Reset index after concatenation
)
train_static

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
0,0,NaN,NaN,1917.6000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,None,NaT
1,1,NaN,NaN,3134.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,None,NaT
2,2,NaN,NaN,4937.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,AL,NaT
3,3,NaN,NaN,4643.6000,0.0,0.0,1.0,0.0,2.0,0.0,...,1.0,1.0,NaN,NaN,0.0,0.00,NaN,BO,AL,NaT
4,4,NaN,NaN,3390.2000,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,AL,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,2703450,0.0,176561.36,3675.4001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,428159.66,14346.319,FO,None,NaT
1526655,2703451,0.0,301276.47,7088.6000,6191.6,0.0,0.0,5.0,0.0,0.0,...,0.0,3.0,68098.4,68098.4,68098.4,701247.30,40499.805,FO,None,NaT
1526656,2703452,0.0,14232.40,7788.8003,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,24002.00,NaN,BO,None,NaT
1526657,2703453,0.0,197371.58,1195.4000,2827.2,0.0,0.0,36.0,0.0,0.0,...,0.0,4.0,46806.6,46806.6,46806.6,440145.30,5654.400,BO,None,NaT


In [13]:
dataPath = '/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train'
trian_sta_path = [
    dataPath + "/train_static_cb_0.parquet",
] 
train_static_cb = pd.concat(
    [pd.read_parquet(file_path).pipe(set_table_dtypes) for file_path in trian_sta_path],
    axis=0,  # Concatenate along the rows (vertically)
    ignore_index=True  # Reset index after concatenation
)
train_static_cb

,case_id,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,...,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
0,357,NaT,NaT,NaT,1988-04-01,NaN,NaT,NaT,NaN,NaN,...,6.0,6301.4000,None,2019-01-25,NaT,NaT,None,NaN,NaN,NaN
1,381,NaT,NaT,NaT,1973-11-01,NaN,NaT,NaT,NaN,NaN,...,6.0,4019.6000,None,2019-01-25,NaT,NaT,None,NaN,NaN,NaN
2,388,NaT,NaT,NaT,1989-04-01,NaN,1989-04-01,NaT,6.0,8.0,...,6.0,14548.0000,None,2019-01-28,NaT,NaT,None,NaN,3.0,5.0
3,405,NaT,NaT,NaT,1974-03-01,NaN,1974-03-01,NaT,0.0,0.0,...,6.0,10498.2400,None,2019-01-21,NaT,NaT,None,NaN,2.0,0.0
4,409,NaT,NaT,NaT,1993-06-01,NaN,1993-06-01,NaT,2.0,3.0,...,7.0,6344.8804,None,2019-01-21,NaT,NaT,None,NaN,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500471,2703450,NaT,NaT,2018-01-11,NaT,52863.59,1960-01-01,NaT,0.0,0.0,...,NaN,NaN,None,NaT,NaT,2020-10-19,None,NaN,1.0,1.0
1500472,2703451,NaT,NaT,2005-06-15,NaT,324608.52,1950-11-01,NaT,0.0,0.0,...,NaN,NaN,None,NaT,NaT,2020-10-19,None,NaN,1.0,2.0
1500473,2703452,NaT,NaT,NaT,NaT,102738.76,1977-08-01,NaT,2.0,2.0,...,NaN,NaN,None,NaT,NaT,2020-10-19,None,NaN,0.0,4.0
1500474,2703453,NaT,NaT,2008-02-15,NaT,212683.29,1950-02-01,NaT,2.0,2.0,...,NaN,NaN,None,NaT,NaT,2020-10-17,None,NaN,2.0,1.0


In [14]:
colll = train_static_cb.columns.tolist()
for c in tqdm(colll):
    if c not in feature_list:
        train_static_cb.drop(columns=c, inplace=True)
train_static_cb.head()

100%|██████████| 53/53 [00:03<00:00, 14.72it/s]


,case_id,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,firstquarter_103L,for3years_128L,for3years_504L,...,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,secondquarter_766L,thirdquarter_1082L
0,357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,6301.4000,NaN,NaN
1,381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,4019.6000,NaN,NaN
2,388,NaN,6.0,8.0,2.0,10.0,4.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,14548.0000,3.0,5.0
3,405,NaN,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,10498.2400,2.0,0.0
4,409,NaN,2.0,3.0,0.0,3.0,1.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.0,6344.8804,0.0,4.0


In [ ]:
# train_static_cb_g = train_static_cb.groupby('case_id')['pmtssum_45A'].mean()
# train_static_cb_g = train_static_cb_g.reset_index()
# train_static_cb_g.head()

In [15]:
colll = train_static.columns.tolist()
for c in tqdm(colll):
    if c not in feature_list:
        train_static.drop(columns=c, inplace=True)
train_static

100%|██████████| 168/168 [00:30<00:00,  5.46it/s]


,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A
0,0,NaN,NaN,1917.6000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.00,NaN
1,1,NaN,NaN,3134.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.00,NaN
2,2,NaN,NaN,4937.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.00,NaN
3,3,NaN,NaN,4643.6000,0.0,0.0,1.0,0.0,2.0,0.0,...,0.0,NaN,NaN,1.0,1.0,NaN,NaN,0.0,0.00,NaN
4,4,NaN,NaN,3390.2000,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,2703450,0.0,176561.36,3675.4001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,428159.66,14346.319
1526655,2703451,0.0,301276.47,7088.6000,6191.6,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,NaN,0.0,3.0,68098.4,68098.4,68098.4,701247.30,40499.805
1526656,2703452,0.0,14232.40,7788.8003,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24002.00,NaN
1526657,2703453,0.0,197371.58,1195.4000,2827.2,0.0,0.0,36.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,46806.6,46806.6,46806.6,440145.30,5654.400


In [16]:
merged_df = pd.merge(train_base, train_static, on='case_id', how='left')
merged_df = pd.merge(merged_df, train_static_cb, on='case_id', how='left')
merged_df

,case_id,target,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,...,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,secondquarter_766L,thirdquarter_1082L
0,0,0,NaN,NaN,1917.6000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,NaN,NaN,3134.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,4937.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0,NaN,NaN,4643.6000,0.0,0.0,1.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,NaN,NaN,3390.2000,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,2703450,0,0.0,176561.36,3675.4001,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,12155.0,NaN,12.0,NaN,NaN,NaN,1.0,1.0
1526655,2703451,0,0.0,301276.47,7088.6000,6191.6,0.0,0.0,5.0,0.0,...,NaN,NaN,22904.6,NaN,12.0,NaN,NaN,NaN,1.0,2.0
1526656,2703452,0,0.0,14232.40,7788.8003,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.0
1526657,2703453,0,0.0,197371.58,1195.4000,2827.2,0.0,0.0,36.0,0.0,...,NaN,NaN,15792.4,NaN,14.0,NaN,NaN,NaN,2.0,1.0


# Part app

In [17]:
dataPath = '/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train'
applprev_path = [
    dataPath + "/train_applprev_1_1.parquet",
    dataPath + "/train_applprev_1_0.parquet",
] 
train_applprev = pd.concat(
    [pd.read_parquet(file_path).pipe(set_table_dtypes) for file_path in applprev_path],
    axis=0,  # Concatenate along the rows (vertically)
    ignore_index=True  # Reset index after concatenation
)
train_applprev = train_applprev[train_applprev['case_id'] < 1526660]
train_applprev

,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,...,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,40704,0.0,7204.6000,NaT,NaN,P94_109_143,NaN,2018-11-20,NaN,0.0,...,0,NaN,12.0,P46_145_78,a55475b1,P198_131_9,P94_109_143,NaN,D,12.0
1,40734,0.0,3870.2000,NaT,NaN,P94_109_143,NaN,2019-12-26,NaN,0.0,...,0,NaN,18.0,P149_40_170,a55475b1,P45_84_106,P94_109_143,NaN,D,18.0
2,40737,0.0,2324.4001,NaT,1.0,a55475b1,0.0,2014-07-17,NaN,0.0,...,0,0.00,18.0,P46_145_78,a55475b1,a55475b1,a55475b1,NaN,D,18.0
3,40791,0.0,2320.8000,NaT,1.0,a55475b1,0.0,2014-12-28,NaN,0.0,...,1,0.00,12.0,P60_146_156,a55475b1,a55475b1,a55475b1,NaN,D,12.0
4,40791,0.0,2541.2000,NaT,1.0,a55475b1,0.0,2014-12-28,NaN,0.0,...,2,0.00,24.0,P177_117_192,a55475b1,a55475b1,a55475b1,NaN,D,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4956070,1526658,0.0,4801.4000,2016-06-25,NaN,a55475b1,0.0,2016-06-25,NaN,0.0,...,8,0.00,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,K,24.0
4956071,1526658,0.0,5322.2000,2019-01-10,NaN,a55475b1,NaN,2019-01-10,NaN,0.0,...,0,148956.61,36.0,a55475b1,a55475b1,P94_109_143,a55475b1,NaN,A,36.0
4956072,1526658,0.0,8083.4000,NaT,NaN,P94_109_143,NaN,2018-05-07,NaN,0.0,...,2,NaN,5.0,a55475b1,a55475b1,P94_109_143,P94_109_143,NaN,D,5.0
4956073,1526658,0.0,8320.2000,NaT,NaN,P94_109_143,NaN,2018-01-02,NaN,0.0,...,4,NaN,24.0,a55475b1,a55475b1,P94_109_143,P94_109_143,NaN,D,24.0


In [18]:
colll = train_applprev.columns.tolist()
for c in tqdm(colll):
    if c not in feature_list:
        train_applprev.drop(columns=c, inplace=True)
train_applprev.head()

100%|██████████| 41/41 [00:12<00:00,  3.18it/s]


,case_id,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
0,40704,0.0,7204.6000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,54000.0,NaN,0.0,40000.0,NaN,NaN,12.0,NaN,12.0
1,40734,0.0,3870.2000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,50000.0,NaN,0.0,50000.0,NaN,NaN,18.0,NaN,18.0
2,40737,0.0,2324.4001,1.0,0.0,NaN,0.0,NaN,NaN,NaN,30000.0,0.0,0.0,16000.0,NaN,0.0,18.0,NaN,18.0
3,40791,0.0,2320.8000,1.0,0.0,NaN,0.0,NaN,NaN,NaN,27830.0,0.0,0.0,16000.0,NaN,0.0,12.0,NaN,12.0
4,40791,0.0,2541.2000,1.0,0.0,NaN,0.0,NaN,NaN,NaN,58239.8,0.0,0.0,22000.0,NaN,0.0,24.0,NaN,24.0


In [19]:
train_applprev.columns

Index(['case_id', 'actualdpd_943P', 'annuity_853A', 'byoccupationinc_3656910L',
       'childnum_21L', 'credacc_actualbalance_314A', 'credacc_credlmt_575A',
       'credacc_maxhisbal_375A', 'credacc_minhisbal_90A',
       'credacc_transactions_402L', 'credamount_590A', 'currdebt_94A',
       'downpmt_134A', 'mainoccupationinc_437A', 'maxdpdtolerance_577P',
       'outstandingdebt_522A', 'pmtnum_8L', 'revolvingaccount_394A',
       'tenor_203L'],
      dtype='object')

In [20]:
columns_to_mean = ['actualdpd_943P', 'annuity_853A', 'byoccupationinc_3656910L',
       'childnum_21L', 'credacc_actualbalance_314A', 'credacc_credlmt_575A',
       'credacc_maxhisbal_375A', 'credacc_minhisbal_90A',
       'credacc_transactions_402L', 'credamount_590A', 'currdebt_94A',
       'downpmt_134A', 'mainoccupationinc_437A', 'maxdpdtolerance_577P',
       'outstandingdebt_522A', 'pmtnum_8L', 'revolvingaccount_394A',
       'tenor_203L']

train_applprev = train_applprev.groupby('case_id')[columns_to_mean].mean()
train_applprev

,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
case_id,,,,,,,,,,,,,,,,,,
2,0.0,1161.300000,NaN,0.0,NaN,0.000000,NaN,NaN,NaN,13000.000000,NaN,0.0,8200.000000,NaN,NaN,18.000000,NaN,18.000000
3,0.0,6140.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,59999.800000,NaN,0.0,11000.000000,NaN,NaN,12.000000,NaN,12.000000
4,0.0,2556.600000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,40000.000000,NaN,0.0,16000.000000,NaN,NaN,24.000000,NaN,24.000000
5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62000.000000,NaN,NaN,NaN,NaN,NaN
6,0.0,2357.933333,1.00,0.0,NaN,0.000000,NaN,NaN,NaN,21786.666667,0.000000,0.0,28666.666667,NaN,0.000000,15.333333,NaN,15.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526655,0.0,928.000000,NaN,0.0,10699.0,11992.000000,-7289.0,-8089.0,0.0,14464.000000,5069.066667,0.0,48000.000000,0.00,5069.066667,13.500000,754761340.0,13.500000
1526656,0.0,3720.900000,1.00,0.0,NaN,0.000000,NaN,NaN,NaN,47300.000000,1994.153000,0.0,34500.000000,26.00,2078.400150,25.000000,NaN,25.000000
1526657,0.0,3357.200100,1.00,1.5,83487.4,27829.133333,0.0,0.0,0.0,70495.133333,0.000000,0.0,51333.333333,0.00,0.000000,15.000000,741869000.0,15.000000


In [21]:
merged_df = pd.merge(merged_df, train_applprev, on='case_id', how='left')
merged_df

,case_id,target,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,...,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
0,0,0,NaN,NaN,1917.6000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,NaN,NaN,3134.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,4937.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,13000.0,NaN,0.0,8200.0,NaN,NaN,18.0,NaN,18.0
3,3,0,NaN,NaN,4643.6000,0.0,0.0,1.0,0.0,2.0,...,NaN,59999.8,NaN,0.0,11000.0,NaN,NaN,12.0,NaN,12.0
4,4,1,NaN,NaN,3390.2000,0.0,0.0,1.0,0.0,0.0,...,NaN,40000.0,NaN,0.0,16000.0,NaN,NaN,24.0,NaN,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,2703450,0,0.0,176561.36,3675.4001,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526655,2703451,0,0.0,301276.47,7088.6000,6191.6,0.0,0.0,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526656,2703452,0,0.0,14232.40,7788.8003,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526657,2703453,0,0.0,197371.58,1195.4000,2827.2,0.0,0.0,36.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Part Tax

In [22]:
dataPath = '/kaggle/input/home-credit-credit-risk-model-stability/parquet_files/train'
tax_path = [
    dataPath + "/train_tax_registry_a_1.parquet",
    dataPath + "/train_tax_registry_b_1.parquet",
] 
train_tax = pd.concat(
    [pd.read_parquet(file_path).pipe(set_table_dtypes) for file_path in tax_path],
    axis=0,  # Concatenate along the rows (vertically)
    ignore_index=True  # Reset index after concatenation
)
train_tax = train_tax[train_tax['case_id'] < 1526660]
train_tax

,case_id,amount_4527230A,name_4527232M,num_group1,recorddate_4527225D,amount_4917619A,deductiondate_4917603D,name_4917606M
0,28631,711.0000,f980a1ea,3,2019-09-13,NaN,NaT,NaN
1,28631,1946.0000,f980a1ea,2,2019-09-13,NaN,NaT,NaN
2,28631,2600.0000,f980a1ea,1,2019-09-13,NaN,NaT,NaN
3,28631,3616.4001,f980a1ea,0,2019-09-13,NaN,NaT,NaN
4,28632,400.0000,5f9b74f5,6,2019-09-13,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...
3710719,1027774,NaN,NaN,2,NaT,13676.0,2020-07-10,cfabf29d
3710720,1027774,NaN,NaN,3,NaT,13676.0,2020-08-13,cfabf29d
3710721,1027774,NaN,NaN,4,NaT,13676.0,2020-09-08,cfabf29d
3710722,1027774,NaN,NaN,6,NaT,13676.0,2020-10-07,cfabf29d


In [23]:
colll = train_tax.columns.tolist()
for c in tqdm(colll):
    if c not in feature_list:
        train_tax.drop(columns=c, inplace=True)
train_tax.head()

100%|██████████| 8/8 [00:00<00:00, 32.65it/s]


,case_id,amount_4527230A,amount_4917619A
0,28631,711.0000,NaN
1,28631,1946.0000,NaN
2,28631,2600.0000,NaN
3,28631,3616.4001,NaN
4,28632,400.0000,NaN


In [24]:
train_tax.columns

Index(['case_id', 'amount_4527230A', 'amount_4917619A'], dtype='object')

In [25]:
columns_to_mean = ['amount_4527230A','amount_4917619A']

train_tax = train_tax.groupby('case_id')[columns_to_mean].mean()
train_tax

,amount_4527230A,amount_4917619A
case_id,,
28631,2218.350025,NaN
28632,1405.180010,NaN
28633,151.600000,NaN
28635,5549.800300,NaN
28636,1773.942871,NaN
...,...,...
1526652,4975.033400,NaN
1526653,499.400000,NaN
1526654,1487.000005,NaN


In [26]:
merged_df = pd.merge(merged_df, train_tax, on='case_id', how='left')
merged_df

,case_id,target,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,...,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L,amount_4527230A,amount_4917619A
0,0,0,NaN,NaN,1917.6000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,NaN,NaN,3134.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,NaN,NaN,4937.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,0.0,8200.0,NaN,NaN,18.0,NaN,18.0,NaN,NaN
3,3,0,NaN,NaN,4643.6000,0.0,0.0,1.0,0.0,2.0,...,NaN,0.0,11000.0,NaN,NaN,12.0,NaN,12.0,NaN,NaN
4,4,1,NaN,NaN,3390.2000,0.0,0.0,1.0,0.0,0.0,...,NaN,0.0,16000.0,NaN,NaN,24.0,NaN,24.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,2703450,0,0.0,176561.36,3675.4001,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526655,2703451,0,0.0,301276.47,7088.6000,6191.6,0.0,0.0,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526656,2703452,0,0.0,14232.40,7788.8003,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526657,2703453,0,0.0,197371.58,1195.4000,2827.2,0.0,0.0,36.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Done

In [25]:
merged_df = set_table_dtypes(merged_df)

In [27]:
merged_df.to_parquet('/kaggle/working/train_fea_final_v4.parquet', index=False)